In [82]:
using Pkg
Pkg.add("WebIO")

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [0f1e0344] + WebIO v0.8.20
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [85]:
using WebIO
WebIO.install_jupyter_nbextension(WebIO.find_jupyter_cmd(force_conda_jupyter=true))

LoadError: The WebIO Jupyter extension must be installed using Python or Conda. See https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/ for more information.

In [1]:
using CSV
using DataFrames
using StatsBase
using MultivariateStats 
using PlotlyJS,MLJ


┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1662
┌ Info: Precompiling PlotlyJS [f0f68f2c-4968-5e81-91da-67840de0976a]
└ @ Base loading.jl:1662


WebIO._IJuliaInit()

┌ Info: Precompiling MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
└ @ Base loading.jl:1662


In [104]:
# read from csv file
csv_reader = CSV.File("./Data/USArrests.csv")
println(typeof(csv_reader))

CSV.File


In [105]:
df = DataFrame(csv_reader)
first(df,5)

Row,Column1,Murder,Assault,UrbanPop,Rape
,String15,Float64,Int64,Int64,Float64
1,Alabama,13.2,236,58,21.2
2,Alaska,10.0,263,48,44.5
3,Arizona,8.1,294,80,31.0
4,Arkansas,8.8,190,50,19.5
5,California,9.0,276,91,40.6


In [106]:
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Column1,,Alabama,,Wyoming,0,String15
2,Murder,7.788,0.8,7.25,17.4,0,Float64
3,Assault,170.76,45,159.0,337,0,Int64
4,UrbanPop,65.54,32,66.0,91,0,Int64
5,Rape,21.232,7.3,20.1,46.0,0,Float64


In [107]:
newdf = df[!, Not("Column1")]

Row,Murder,Assault,UrbanPop,Rape
,Float64,Int64,Int64,Float64
1,13.2,236,58,21.2
2,10.0,263,48,44.5
3,8.1,294,80,31.0
4,8.8,190,50,19.5
5,9.0,276,91,40.6
6,7.9,204,78,38.7
7,3.3,110,77,11.1
8,5.9,238,72,15.8
9,15.4,335,80,31.9


In [126]:
df1 = Matrix(newdf) #convert dataframe to matrix
dt_std = standardize(ZScoreTransform, df1, dims=1)
dt_std = dt_std' #take transpose because julia fit function needs transpose

4×50 adjoint(::Matrix{Float64}) with eltype Float64:
  1.24256      0.507862  0.0716334  …  -0.479393  -1.19113    -0.226839
  0.782839     1.10682   1.4788        -1.07706   -1.41305    -0.117114
 -0.520907    -1.21176   0.99898       -1.83354    0.0317794  -0.382735
 -0.00341647   2.4842    1.04288       -1.27392   -1.11377    -0.601299

In [139]:
M = fit(PCA,dt_std;maxoutdim=4)

PCA(indim = 4, outdim = 4, principalratio = 1.0)

Pattern matrix (unstandardized loadings):
─────────────────────────────────────────────
        PC1        PC2        PC3         PC4
─────────────────────────────────────────────
1  0.843976  -0.416035   0.20376    0.270371
2  0.918443  -0.187021   0.160119  -0.309592
3  0.438117   0.868328   0.225724   0.0557533
4  0.855839   0.16646   -0.488319   0.0370741
─────────────────────────────────────────────

Importance of components:
──────────────────────────────────────────────────────────────────
                               PC1       PC2        PC3        PC4
──────────────────────────────────────────────────────────────────
SS Loadings (Eigenvalues)  2.48024  0.989765  0.356563   0.17343
Variance explained         0.62006  0.247441  0.0891408  0.0433575
Cumulative variance        0.62006  0.867502  0.956642   1.0
Proportion explained       0.62006  0.247441  0.0891408  0.0433575
Cumulative proportion      0.62006  0.867502  0.956642

In [140]:
dt_transformed = MultivariateStats.transform(M, dt_std)
pca_mat = dt_transformed'
pca_mat[:,1] = -pca_mat[:,1]
pca_mat[:,2] = -pca_mat[:,2]
pca_mat

50×4 adjoint(::Matrix{Float64}) with eltype Float64:
  0.97566     1.122      -0.439804    0.154697
  1.93054     1.06243     2.0195     -0.434175
  1.74544    -0.73846     0.0542302  -0.826264
 -0.139999    1.10854     0.113422   -0.180974
  2.49861    -1.52743     0.592541   -0.338559
  1.49934    -0.97763     1.084       0.00145016
 -1.34499    -1.07798    -0.636793   -0.117279
  0.0472298  -0.322089   -0.71141    -0.873113
  2.98276     0.0388342  -0.571032   -0.095317
  1.62281     1.26609    -0.339018    1.06597
 -0.903484   -1.55468     0.0502715   0.893733
 -1.62332     0.208853    0.25719    -0.494088
  1.36505    -0.674988   -0.670686   -0.120795
  ⋮                                  
 -0.855091   -1.47698    -1.35618    -0.607403
  1.30745     1.91397    -0.297517   -0.130145
 -1.9678      0.815068    0.385381   -0.108471
  0.989694    0.851605    0.186193    0.646303
  1.34152    -0.408335   -0.487123    0.636731
 -0.545032   -1.45672     0.290776   -0.0814867
 -2.77326     

In [142]:
load = MultivariateStats.loadings(M::PCA)
pca_df = DataFrame(pca_mat[:,1:2], :auto)
pca_df = rename!(pca_df,[:PC1,:PC2])
pca_df = insertcols!(pca_df, 1, :city => df[:,"Column1"])
proj = MultivariateStats.projection(M::PCA)
pca_df

Row,city,PC1,PC2
,String15,Float64,Float64
1,Alabama,0.97566,1.122
2,Alaska,1.93054,1.06243
3,Arizona,1.74544,-0.73846
4,Arkansas,-0.139999,1.10854
5,California,2.49861,-1.52743
6,Colorado,1.49934,-0.97763
7,Connecticut,-1.34499,-1.07798
8,Delaware,0.0472298,-0.322089
9,Florida,2.98276,0.0388342


In [143]:
proj[:,1] = -1 * proj[:,1]
proj[:,2] = -1 * proj[:,2]
proj

4×4 Matrix{Float64}:
 0.535899   0.418181  -0.341233   0.649228
 0.583184   0.187986  -0.268148  -0.743407
 0.278191  -0.872806  -0.378016   0.133878
 0.543432  -0.167319   0.817778   0.0890243

In [77]:
matrx  = [[0.535899,0.583184,0.278191,0.543432] [0.418181,0.187986,-0.872806,-0.167319] [-0.341233,-0.268148,-0.378016,0.817778] [0.649228,-0.743407,0.133878,0.089024]]
matrx' * matrx

4×4 Matrix{Float64}:
  1.0        -6.5611e-8    3.8341e-8   -2.8685e-7
 -6.5611e-8   1.0         -3.91387e-7  -2.22358e-7
  3.8341e-8  -3.91387e-7   1.0         -1.07526e-6
 -2.8685e-7  -2.22358e-7  -1.07526e-6   1.0

In [144]:
features = [:Murder, :Assault, :UrbanPop, :Rape]
plot(pca_df, x=:PC1, y=:PC2, mode="markers",text=:city,
    Layout(
        shapes=[
            line(x0=0, y0=0, x1=proj[1, i], y1=proj[2, i])
            for i in 1:length(features)
        ],
        annotations=[
            attr(
                x=proj[1, i], y=proj[2, i], text=features[i],
                xanchor="center", yanchor="bottom"
            )
            for i in 1:length(features)
        ]
        )) 
#Hover over the dots to see the corresponding city label


data: [
  "scatter with fields mode, text, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, shapes, template, xaxis, and yaxis"

In [159]:
p = principalvars(M::PCA)

4-element Vector{Float64}:
 2.4802415791494927
 0.9897651525398412
 0.3565631805808296
 0.1734300877298358

In [160]:
# Standard deviation of the four principal components
sqrt.(p)

4-element Vector{Float64}:
 1.5748782743912282
 0.9948694148177645
 0.5971291155025265
 0.41644938195396064

In [162]:
var_expl = [[p[1]/sum(p)] [p[2]/sum(p)] [p[3]/sum(p)] [p[4]/sum(p)] ]

1×4 Matrix{Float64}:
 0.62006  0.247441  0.0891408  0.0433575

In [ ]:
plot()